In [1]:
from bs4 import BeautifulSoup
import spotipy
import pandas as pd
import numpy as np
import requests
import random
from spotipy.oauth2 import SpotifyClientCredentials
import time
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle

In [2]:
scaler_final = pickle.load(open("scaler_final.pkl", "rb"))

In [3]:
Model_predictor = pickle.load(open("KMeans_final.pkl", "rb"))

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="a3a2927271164d2786c28785f365ae9b",
    client_secret="5e240c22b70140ac8b613a2529b0ccd2"))

In [6]:
billboard = pd.read_csv("../Labs/Data/Billboard.csv")
sp_data = pd.read_csv("../Labs/Data/Spotify_Data.csv")
full_data = pd.read_csv("../Labs/Data/Full_data.csv")

In [41]:
def song_search(input_song):
    '''Lets the user input song and function will find the artist'''
    result = sp.search(q=input_song, limit = 1)
    uri_input_song = result['tracks']['items'][0]['uri']
    features_input_song = sp.audio_features(uri_input_song)[0]
    print("Ah yes!", input_song, "by", result['tracks']['items'][0]['artists'][0]['name'])
    return input_song

    if input_song in billboard.values:
        print("That is a hot song!")
        random_song = random.choice(billboard["Title"])
        while random_song == input_song:
            random_song = random.choice(billboard["Title"])
        print("Let me recommend you another hot song! Check out:", random_song, "by",billboard.loc[billboard["Title"] == random_song, 'Artist'].iloc[0])
    elif input_song not in billboard.values:
        new_song = input_song
        print("Checking our song libraries for recommendation...")
        return new_song

def collect_features(new_song): 
    global recommend_song
    result = sp.search(q=new_song, limit = 1)
    uri_new_song = result['tracks']['items'][0]['uri']
    f_song = []
    features_new_song = sp.audio_features(uri_new_song)[0]
    f_song.append(features_new_song)
    song_features = pd.DataFrame(f_song, columns=["danceability","energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])
    song_scaled = scaler_final.transform(song_features)
    cluster = int(Model_predictor.predict(song_scaled))
    filtered_df = full_data["clusters"] == cluster
    filtered_data_clusters = full_data[filtered_df]
    recommend_song = filtered_data_clusters.sample()
    print("We found this:", recommend_song.loc[:,"song"].iloc[0], 'by', recommend_song.loc[:,"artist"].iloc[0])
    print("Enjoy!")

In [35]:
recommend_song

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,clusters,uri,song,artist
10973,0,0.426,0.584,1,-5.293,1,0.296,0.0383,0.0,0.188,0.272,193.437,10,spotify:track:5R9a4t5t5O0IsznsrKPVro,Single Ladies (Put a Ring on It),Beyoncé


In [42]:
def main():
    scaler = StandardScaler()
    input_song = str(input("Hello! Please enter a song so I can recommend you one: "))
    song_search(input_song)
    if input_song in billboard.values:
        print("That is a hot song!")
        random_song = random.choice(billboard["Title"])
        while random_song == input_song:
            random_song = random.choice(billboard["Title"])
        print("Let me recommend you another hot song! Check out:", random_song, "by",billboard.loc[billboard["Title"] == random_song, 'Artist'].iloc[0])
        print("Enjoy!")
    elif input_song not in billboard.values:
        new_song = input_song
        print("Checking our song libraries for recommendation...")
        collect_features(new_song)
        


# Gnod Song Recommender Prototype

Song recomendation based on user input

In [52]:
main()

Hello! Please enter a song so I can recommend you one: Californication
Ah yes! Californication by Red Hot Chili Peppers
Checking our song libraries for recommendation...
We found this: Guns And Roses by Lana Del Rey
Enjoy
